In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import stanfordnlp
from textblob import TextBlob

from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB 
from sklearn.naive_bayes import MultinomialNB 


# filepath = '../datasets/news_cleaned_2018_02_13.csv'
# nlinesfile = 9408908

# cols = ['type', 'title']

# nlinesrandomsample = 5000000
# lines2skip = np.random.choice(np.arange(1,nlinesfile+1), (nlinesfile-nlinesrandomsample), replace=False, )
# df = pd.read_csv(filepath, skiprows=lines2skip, usecols= cols )


# print(f"Inital shape : {df.shape}")

# df_f = df[df['type'] == 'fake']
# df_h = df[df['type'] == 'hate']
# df_b = df[df['type'] == 'bias']
# df_p = df[df['type'] == 'political']
# df_r = df[df['type'] == 'reliable']

# df = pd.concat([df_f, df_r, df_h, df_b, df_p], axis = 0)

In [12]:
df.index = range(0, len(df.index))

df.sample(10)

print(f"Subsection df shape : {df.shape}")

# df['type'].value_counts(normalize = True)

df['label'] = df['type'].map({'reliable' : 0, 'political' : 0, 'bias' : 1, 'fake' : 1, 'hate' : 1})

In [13]:
df['label'].value_counts(normalize = True)

0    0.628708
1    0.371292
Name: label, dtype: float64

In [14]:
df.isna().mean()

type     0.000000
title    0.000085
label    0.000000
dtype: float64

In [15]:
df.dropna(inplace = True)

In [16]:
def tokenize(x):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(x)

df['tokens'] = df['title'].map(tokenize)

def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])

df['lemma'] = df['tokens'].map(lemmatize)
    


In [17]:
tf = TfidfVectorizer(max_df=0.8, min_df = 3, stop_words = 'english', ngram_range=(1,2))

In [18]:
X = tf.fit_transform(df['lemma'])

In [19]:
X.shape

(3018995, 855274)

In [20]:
y = df['label']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42, stratify = y)

In [22]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
print(nb.score(X_train, y_train))
print(nb.score(X_test, y_test))

0.8552462939097607
0.8223899601059425


In [23]:
rf = RandomForestClassifier()
rf_grid = {
    'n_estimators' : [5, 25, 50, 100],
    'max_depth' : [2, 4, 6, 8],
    'max_features' : [None, 3, 6]
}

rs = RandomizedSearchCV(rf, rf_grid, n_iter=25, n_jobs=-1, verbose=1)

In [ ]:
rs.fit(X_train, y_train)
print(f"The training score is {rs.score(X_train, y_train)}")
print(f"The testing score is {rs.score(X_test, y_test)}")

Fitting 3 folds for each of 25 candidates, totalling 75 fits


C:\Users\Ishan\Anaconda3\envs\nltktb\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
